In [1]:
import numpy as np
import os
import pandas as pd
import requests
import hmac
import hashlib
import time
from urllib.parse import quote
import json
import pyodbc
import statsmodels.api as sm
from datetime import datetime, UTC
from pandas.tseries.offsets import QuarterBegin
from pandas.tseries.offsets import MonthEnd
from statsmodels.api import OLS
from statsmodels.tools.tools import add_constant
from scipy.stats import norm
from scipy.optimize import root_scalar
from scipy.optimize import brentq
from scipy.stats import multivariate_normal
from statsmodels.api import OLS
from statsmodels.tools.tools import add_constant
import matplotlib.pyplot as plt

In [2]:
# set working directory
os.chdir('../')
# print working directory
os.getcwd()

"c:\\Users\\XiaoA1\\OneDrive - Moody's\\COE - Credit Analytics-SQ - Unified ST Framework - Documents\\Sovereign Macro\\2 - Research Code"

In [3]:
version = '052025'
vintage = '202506'

# Download Data from DataBuffet

In [4]:
#####
# Setup:
# 1. Store your access key, encryption key.
# Get your keys at:
# https://www.economy.com/myeconomy/api-key-info
ACC_KEY = "FF91D2DD-FB35-4664-BDA5-E165A718941D"
ENC_KEY = "3F2579A0-99EF-487C-B0D9-0ECA49DFC46E"


def get_series(mnemonic, acc_key, enc_key, freq="0", trans="0", vintage='latest'):
    if vintage == 'latest':
        vintage = datetime.now(UTC).strftime("%Y%m")
    
    api_command = f"series?m={quote(mnemonic)}&freq={freq}&trans={trans}&vintage={vintage}"
    url = f"https://api.economy.com/data/v1/{api_command}"
    print(url)
    
    time_stamp = datetime.now(UTC).strftime("%Y-%m-%dT%H:%M:%SZ")
    hash_msg = f"{acc_key}{time_stamp}"
    signature = hmac.new(enc_key.encode(), hash_msg.encode(), hashlib.sha256).hexdigest()
    
    time.sleep(1)
    headers = {
        "AccessKeyId": acc_key,
        "Signature": signature,
        "TimeStamp": time_stamp
    }
    
    response = requests.get(url, headers=headers)
    
    series = response.json()
    print(series.get('description'))
    return series

In [5]:
def create_date_range_fixed(start_date, end_date, freq):
    """
    Create date range and convert to date-only format to avoid timestamp issues
    """
    if freq == 'MONTH':
        dates = pd.date_range(start=start_date, end=end_date, freq='M') + MonthEnd(0)
    else:
        dates = pd.date_range(start=start_date, end=end_date, freq='QE') + QuarterBegin(startingMonth=1) - pd.DateOffset(days=1)
    
    # Convert to date-only format to avoid timestamp issues
    dates = dates.date
    return dates

In [6]:
sov_list = pd.read_excel('Output/1.sov_monthly_summary_'+version+'.xlsx')
sov_list = sov_list[sov_list['Included']==True]
sov_list.loc[sov_list['cinc'] == 'SCG', 'cinc'] = 'SRB'
sov_list.head()

,pid,entityName,cinc,region,Nmonth,Nquarter,cdsiedf5 unique value count,start,end,cdsiedf5 unique value count (%),Included
1,S10159,"AUSTRALIA, GOVERNMENT OF",AUS,Australia,233,78,233,2006-01-31,2025-05-31,1.0,True
2,S10539,"AUSTRIA, GOVERNMENT OF",AUT,Europe,233,78,233,2006-01-31,2025-05-31,1.0,True
3,S10368,"BELGIUM, GOVERNMENT OF",BEL,Europe,233,78,233,2006-01-31,2025-05-31,1.0,True
4,S10540,"BULGARIA, GOVERNMENT OF",BGR,Eastern Europe,233,78,233,2006-01-31,2025-05-31,1.0,True
5,S10367,"BAHRAIN, GOVERNMENT OF",BHR,Middle East,233,78,233,2006-01-31,2025-05-31,1.0,True


In [7]:
mnemonic_existing = pd.read_csv("Output/2.sov_mv_mnemonics.csv")
sov_list = pd.merge(sov_list, mnemonic_existing, on=['cinc', 'entityName'], how='left')

In [8]:
mv_types = [
	"GDP",
	"Unemployment Rate",
	"Equity",
	"FX",
	"Inflation",
    "Government 10Y Bond Rate",
    "Government Consumption",
    "Net Exports",
	"Debt to GDP Ratio",
    "Monetary Policy Rate"
]

mv_types_global = [
    "Commodity Index",
    "Oil Price"
]

mnemonics_general = [
    "FGDPD$Q",
    "FLBRQ",
    "FSTOCKPQ",
    "FTFXIUSAQ",
    "FCPIQ",
    "FRGT10YQ",
    "FGD$Q",
    "FNETEXGSD$Q",
    "FGGDEBTGDPQ",
    "FRMPOLQ"
]

mnemonics_global = [
    "FPCFPI.IWRLD",
    "FCPWTI.IUSA"
]

# Create new columns if they don't exist
for col in mv_types + mv_types_global:
    if col not in sov_list.columns:
        sov_list[col] = pd.NA
        sov_list[col] = sov_list[col].astype('string')

scenario_list = ['Baseline', 'S1', 'S3', 'S4']

In [9]:
data_all = pd.read_csv('Output/2_mv_raw_data.csv')
for scenario in scenario_list:
    if scenario == 'Baseline':
        scenario_code = ''
    else:
        scenario_code = "_" + scenario
    for mnemonic in mnemonics_global:
        mnemonic_scenario = mnemonic.split(".")[0] + scenario_code + '.' + mnemonic.split(".")[1]
        if mnemonic_scenario in data_all['ma.ticker'].values:
            continue
        my_data = get_series(mnemonic_scenario, ACC_KEY, ENC_KEY, vintage=vintage)
        if my_data['data'] == None:
            continue
        start_date = datetime.strptime(my_data['startDate'][:10], "%Y-%m-%d")
        end_date = datetime.strptime(my_data['endDate'][:10], "%Y-%m-%d")
        dates = create_date_range_fixed(start_date, end_date, my_data['data']['freq'])
        data = pd.DataFrame({
            'yyyyqq': dates,
            'scenario': scenario,
            'cinc': 'Global',
            'ma.ticker': mnemonic_scenario,
            'value': pd.to_numeric(my_data['data']['data'])
        })
        data_all = pd.concat([data_all, data], ignore_index = True)

    for cinc in sov_list['cinc']:
        for mv_type, mnemonic in zip(mv_types, mnemonics_general):
            mnemonic_scenario = mnemonic + scenario_code + ".I" + cinc
            if mnemonic_scenario in data_all['ma.ticker'].values:
                continue
            my_data = get_series(mnemonic_scenario, ACC_KEY, ENC_KEY, vintage=vintage)
            if my_data['data'] == None:
                continue
            start_date = datetime.strptime(my_data['startDate'][:10], "%Y-%m-%d")
            end_date = datetime.strptime(my_data['endDate'][:10], "%Y-%m-%d")
            dates = create_date_range_fixed(start_date, end_date, my_data['data']['freq'])
            data = pd.DataFrame({
                'yyyyqq': dates,
                'scenario': scenario,   
                'cinc': cinc,
                'ma.ticker': mnemonic_scenario,
                'value': pd.to_numeric(my_data['data']['data'])
            })
            if scenario == 'Baseline':
                sov_list.loc[sov_list['cinc']==cinc, mv_type] = mnemonic_scenario
            data_all = pd.concat([data_all, data], ignore_index = True)

data_all.to_csv(f'Output/2_mv_raw_data.csv', index = False)

https://api.economy.com/data/v1/series?m=FRMPOLQ.IAUS&freq=0&trans=0&vintage=202506
Baseline Scenario (June 2025): Interest rate: Overnight Target Rate [or Cash Rate], (% p.a., NSA)
https://api.economy.com/data/v1/series?m=FRMPOLQ.IAUT&freq=0&trans=0&vintage=202506
Baseline Scenario (June 2025): Interest rate: Bank Rate - Refinancing Rate - ECB, (% p.a., NSA)
https://api.economy.com/data/v1/series?m=FRMPOLQ.IBEL&freq=0&trans=0&vintage=202506
Baseline Scenario (June 2025): Interest rate: Bank Rate - Refinancing Rate - ECB, (% p.a., NSA)
https://api.economy.com/data/v1/series?m=FRMPOLQ.IBGR&freq=0&trans=0&vintage=202506
Baseline Scenario (June 2025): Interest rate: Nominal Annual Effective Base Interest Rate, (% p.a., NSA)
https://api.economy.com/data/v1/series?m=FRGT10YQ.IBHR&freq=0&trans=0&vintage=202506
None
https://api.economy.com/data/v1/series?m=FRMPOLQ.IBHR&freq=0&trans=0&vintage=202506
Baseline Scenario (June 2025): Interest rate: One Week Deposit Facility, (% p.a., NSA)
https://

In [10]:
for i in range(len(mv_types_global)):
    sov_list[mv_types_global[i]] = mnemonics_global[i]
display(sov_list[['cinc', 'entityName'] + mv_types + mv_types_global].style.set_properties(**{'max-height': 'none', 'max-width': 'none'}))

,cinc,entityName,GDP,Unemployment Rate,Equity,FX,Inflation,Government 10Y Bond Rate,Government Consumption,Net Exports,Debt to GDP Ratio,Monetary Policy Rate,Commodity Index,Oil Price
0,AUS,"AUSTRALIA, GOVERNMENT OF",FGDPD$Q.IAUS,FLBRQ.IAUS,FSTOCKPQ.IAUS,FTFXIUSAQ.IAUS,FCPIQ.IAUS,FRGT10YQ.IAUS,FGD$Q.IAUS,FNETEXGSD$Q.IAUS,FGGDEBTGDPQ.IAUS,FRMPOLQ.IAUS,FPCFPI.IWRLD,FCPWTI.IUSA
1,AUT,"AUSTRIA, GOVERNMENT OF",FGDPD$Q.IAUT,FLBRQ.IAUT,FSTOCKPQ.IAUT,FTFXIUSAQ.IAUT,FCPIQ.IAUT,FRGT10YQ.IAUT,FGD$Q.IAUT,FNETEXGSD$Q.IAUT,FGGDEBTGDPQ.IAUT,FRMPOLQ.IAUT,FPCFPI.IWRLD,FCPWTI.IUSA
2,BEL,"BELGIUM, GOVERNMENT OF",FGDPD$Q.IBEL,FLBRQ.IBEL,FSTOCKPQ.IBEL,FTFXIUSAQ.IBEL,FCPIQ.IBEL,FRGT10YQ.IBEL,FGD$Q.IBEL,FNETEXGSD$Q.IBEL,FGGDEBTGDPQ.IBEL,FRMPOLQ.IBEL,FPCFPI.IWRLD,FCPWTI.IUSA
3,BGR,"BULGARIA, GOVERNMENT OF",FGDPD$Q.IBGR,FLBRQ.IBGR,FSTOCKPQ.IBGR,FTFXIUSAQ.IBGR,FCPIQ.IBGR,FRGT10YQ.IBGR,FGD$Q.IBGR,FNETEXGSD$Q.IBGR,FGGDEBTGDPQ.IBGR,FRMPOLQ.IBGR,FPCFPI.IWRLD,FCPWTI.IUSA
4,BHR,"BAHRAIN, GOVERNMENT OF",FGDPD$Q.IBHR,FLBRQ.IBHR,FSTOCKPQ.IBHR,FTFXIUSAQ.IBHR,FCPIQ.IBHR,nan,FGD$Q.IBHR,FNETEXGSD$Q.IBHR,FGGDEBTGDPQ.IBHR,FRMPOLQ.IBHR,FPCFPI.IWRLD,FCPWTI.IUSA
5,BRA,"BRAZIL, GOVERNMENT OF",FGDPD$Q.IBRA,FLBRQ.IBRA,FSTOCKPQ.IBRA,FTFXIUSAQ.IBRA,FCPIQ.IBRA,FRGT10YQ.IBRA,FGD$Q.IBRA,FNETEXGSD$Q.IBRA,FGGDEBTGDPQ.IBRA,FRMPOLQ.IBRA,FPCFPI.IWRLD,FCPWTI.IUSA
6,CAN,"CANADA, GOVERNMENT OF",FGDPD$Q.ICAN,FLBRQ.ICAN,FSTOCKPQ.ICAN,FTFXIUSAQ.ICAN,FCPIQ.ICAN,FRGT10YQ.ICAN,FGD$Q.ICAN,FNETEXGSD$Q.ICAN,FGGDEBTGDPQ.ICAN,FRMPOLQ.ICAN,FPCFPI.IWRLD,FCPWTI.IUSA
7,CHE,"SWITZERLAND, GOVERNMENT OF",FGDPD$Q.ICHE,FLBRQ.ICHE,FSTOCKPQ.ICHE,FTFXIUSAQ.ICHE,FCPIQ.ICHE,FRGT10YQ.ICHE,FGD$Q.ICHE,FNETEXGSD$Q.ICHE,FGGDEBTGDPQ.ICHE,FRMPOLQ.ICHE,FPCFPI.IWRLD,FCPWTI.IUSA
8,CHL,"CHILE, GOVERNMENT OF",FGDPD$Q.ICHL,FLBRQ.ICHL,FSTOCKPQ.ICHL,FTFXIUSAQ.ICHL,FCPIQ.ICHL,FRGT10YQ.ICHL,FGD$Q.ICHL,FNETEXGSD$Q.ICHL,FGGDEBTGDPQ.ICHL,FRMPOLQ.ICHL,FPCFPI.IWRLD,FCPWTI.IUSA
9,CHN,"CHINA, GOVERNMENT OF",FGDPD$Q.ICHN,FLBRQ.ICHN,FSTOCKPQ.ICHN,FTFXIUSAQ.ICHN,FCPIQ.ICHN,FRGT10YQ.ICHN,FGD$Q.ICHN,FNETEXGSD$Q.ICHN,FGGDEBTGDPQ.ICHN,FRMPOLQ.ICHN,FPCFPI.IWRLD,FCPWTI.IUSA


In [12]:
sov_list[['cinc', 'entityName'] + mv_types + mv_types_global].to_csv("Output/2.sov_mv_mnemonics.csv", index=False)

In [13]:
mnemonics_long = sov_list[['cinc', 'entityName'] + mv_types].melt(
    id_vars=['cinc', 'entityName'],
    var_name='mv_type',
    value_name='mnemonics'
)
mnemonics_long = mnemonics_long.dropna(subset=['mnemonics'])
mnemonics_long.to_csv("Output/2.sov_mv_mnemonics_long.csv", index=False)
mnemonics_long

,cinc,entityName,mv_type,mnemonics
0,AUS,"AUSTRALIA, GOVERNMENT OF",GDP,FGDPD$Q.IAUS
1,AUT,"AUSTRIA, GOVERNMENT OF",GDP,FGDPD$Q.IAUT
2,BEL,"BELGIUM, GOVERNMENT OF",GDP,FGDPD$Q.IBEL
3,BGR,"BULGARIA, GOVERNMENT OF",GDP,FGDPD$Q.IBGR
4,BHR,"BAHRAIN, GOVERNMENT OF",GDP,FGDPD$Q.IBHR
...,...,...,...,...
594,THA,"THAILAND, GOVERNMENT OF",Monetary Policy Rate,FRMPOLQ.ITHA
596,TUR,"TURKEY, GOVERNMENT OF",Monetary Policy Rate,FRMPOLQ.ITUR
597,USA,"UNITED STATES OF AMERICA, GOVERNMENT OF",Monetary Policy Rate,FRMPOLQ.IUSA
598,VNM,"VIETNAM, GOVERNMENT OF",Monetary Policy Rate,FRMPOLQ.IVNM
